In [1]:
from ps4_utils import load_data,load_experiment
from ps4_utils import AbstractGenerativeModel
from ps4_utils import save_submission
from scipy.misc import logsumexp
import numpy as np
data_fn = "datasets-ps4.h5"
MAX_OUTER_ITER = 15

In [2]:
class MixtureModel(AbstractGenerativeModel):
    def __init__(self, CLASSES, NUM_FEATURES, NUM_MIXTURE_COMPONENTS, MAX_ITER=50, EPS=10**(-7)):
        AbstractGenerativeModel.__init__(self, CLASSES, NUM_FEATURES)
        self.num_mixture_components = NUM_MIXTURE_COMPONENTS # list of num_mixture_components (length num_classes)
        self.max_iter = MAX_ITER # max iterations of EM
        self.epsilon = EPS # help with stability, to be used according to hint given at end of pset4.pdf
        self.params = {
            'pi': [np.repeat(1/k,k) for k in self.num_mixture_components],
            'theta': [np.zeros((self.num_features,k)) for k in self.num_mixture_components]
        }
    def pack_params(self, X, class_idx):
        pi,theta = self.fit(X[class_idx],class_idx)
        self.params['pi'][class_idx] = pi
        self.params['theta'][class_idx] = theta
        
    #make classification based on which mixture model gives higher probability to generating point xi
    def classify(self, X):
        P = list()
        pi = self.params['pi']
        theta = self.params['theta']
        for c in range(self.num_classes):
            _,Pc = self.findP(X, pi[c], theta[c])
            P.append(Pc)
        return np.vstack(P).T.argmax(-1)

    # --- E-step
    def updateLatentPosterior(self, X, pi, theta, num_mixture_components): # update the latent posterior
        # YOUR CODE HERE
        return gamma
    
    # --- M-step (1)
    @staticmethod
    def updatePi(gamma): #update the pi component using the posteriors (gammas)
        # YOUR CODE HERE
        return pi_c # class specific pi
    # -- M-step (2)
    @staticmethod
    def updateTheta(X, gamma): #update theta component using posteriors (gammas)
        # YOUR CODE HERE
        return theta_c # class specific theta
    
    @staticmethod
    def findP(X, pi, theta):
        # YOUR CODE HERE
        # --- t: probabilities of x given each component of mixture
        # --- logsumexp(t,axis=1): normalized by factor of probabilities of x over all components of mixture
        return t,logsumexp(t,axis=1)
        
    # --- execute EM procedure
    def fit(self, X, class_idx):
        max_iter = self.max_iter
        eps = self.epsilon
        N = X.shape[0]
        pi = self.params['pi'][class_idx]
        theta = self.params['theta'][class_idx]
        num_mixture_components = self.num_mixture_components[class_idx]
        # INITIALIZE theta
        for i in range(max_iter):
            # YOUR CODE HERE, E-step: gamma = self.updateLatentPosterior
            # YOUR CODE HERE, M-step(1): pi = self.updatePi
            # YOUR CODE HERE, M-step(2): theta = self.updateTheta
        return pi,theta

In [3]:
class NaiveBayesModel(AbstractGenerativeModel):
    def __init__(self, CLASSES, NUM_FEATURES, EPS=10**(-12)):
        AbstractGenerativeModel.__init__(self, CLASSES, NUM_FEATURES)
        self.epsilon = EPS # help with stability
        self.params = {
            'logp': [np.zeros((NUM_FEATURES))] * self.num_classes # estimated log-probabilities
        }
    def pack_params(self, X, class_idx):
        logp = self.fit(X[class_idx])
        self.params['logp'][class_idx] = logp
    def classify(self, X): # naive bayes classifier
        # YOUR CODE HERE
        logp = self.params['logp']
        predictions = np.dot(logp, X.T)
        predictions = np.argmax(predictions, axis = 0)
        return predictions
    def fit(self, X): 
        # YOUR CODE HERE
        estimated_logp = np.log(np.mean(X, axis = 0) + self.epsilon)
        return estimated_logp
    
#     def val(self, X, acc=0, N=0):
#         print self.num_classes
#         for c in range(self.num_classes):
#             predictions = self.classify(X[c])
# #             print('predictions: {}, c: {}'.format(predictions, c))
#             print(np.sum(predictions))
#             acc += np.sum((predictions == c).astype(np.int32))
#             N += X[c].shape[0]
#         return (acc / float(N))

In [4]:
experiment_name = "sentiment_analysis"
# --- SENTIMENT ANALYSIS setup
Xtrain,Xval,num_classes,num_features = load_experiment(data_fn, experiment_name)

# -- build naive bayes model for sentiment analysis
print("SENTIMENT ANALYSIS -- NAIVE BAYES MODEL:")
nbm = NaiveBayesModel(num_classes, num_features)
nbm.train(Xtrain)
print("ACCURACY ON VALIDATION: " + str(nbm.val(Xval)))

# -- build mixture model for sentiment analysis
print("SENTIMENT ANALYSIS -- MIXTURE MODEL:")
for i in range(MAX_OUTER_ITER):
    num_mixture_components =  np.random.randint(2,15,num_classes)
    print("COMPONENTS: " + " ".join(str(i) for i in num_mixture_components))
    mm = MixtureModel(num_classes, num_features, num_mixture_components)
    mm.train(Xtrain)
    print("ACCURACY ON VALIDATION: " + str(mm.val(Xval)))

# submit to kaggle
Xkaggle = load_data(data_fn, experiment_name, "kaggle")
save_submission("nb-{}-submission.csv".format(experiment_name), nbm.classify(Xkaggle))
save_submission("mm-{}-submission.csv".format(experiment_name), mm.classify(Xkaggle))

SENTIMENT ANALYSIS -- NAIVE BAYES MODEL:
ACCURACY ON VALIDATION: 0.72
SENTIMENT ANALYSIS -- MIXTURE MODEL:
COMPONENTS: 6 9


NameError: name 'MixtureModel' is not defined

In [5]:
experiment_name = "mnist"
# --- MNIST DIGIT CLASSIFICATION setup
Xtrain,Xval,num_classes,num_features = load_experiment(data_fn, experiment_name)

# -- build naive bayes model for mnist digit classification
print("MNIST DIGIT CLASSIFICATION -- NAIVE BAYES MODEL:")
nbm = NaiveBayesModel(num_classes, num_features)
nbm.train(Xtrain)
print("ACCURACY ON VALIDATION: " + str(nbm.val(Xval)))

# -- build mixture model for mnist digit classification
print("MNIST DIGIT CLASSIFICATION -- MIXTURE MODEL:")
for i in range(MAX_OUTER_ITER):
    num_mixture_components =  np.random.randint(2,15,num_classes)
    print("COMPONENTS: " + " ".join(str(i) for i in num_mixture_components))
    mm = MixtureModel(num_classes, num_features, num_mixture_components)
    mm.train(Xtrain)
    print("ACCURACY ON VALIDATION: " + str(mm.val(Xval)))
    
# submit to kaggle
Xkaggle = load_data(data_fn, experiment_name, "kaggle")
save_submission("nb-{}-submission.csv".format(experiment_name), nbm.classify(Xkaggle))
save_submission("mm-{}-submission.csv".format(experiment_name), mm.classify(Xkaggle))

MNIST DIGIT CLASSIFICATION -- NAIVE BAYES MODEL:
ACCURACY ON VALIDATION: 0.216
MNIST DIGIT CLASSIFICATION -- MIXTURE MODEL:
COMPONENTS: 2 11 2 8 12 5 3 9 11 7


NameError: name 'MixtureModel' is not defined